In [ ]:
from datetime import datetime
print(datetime.fromtimestamp(1605498402.136237))

In [ ]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
from transformers import AutoModel
from transformers import AutoConfig

In [ ]:
# config = AutoConfig.from_pretrained("prajjwal1/bert-tiny")
config = AutoConfig.from_pretrained("albert-base-v2")
config.hidden_act = 'gelu_fast'
config.torchscript = True
model = AutoModel.from_config(config)
inputs = torch.randint(1000, size=(1, 100)).long()
# model()
# with torch.onnx.select_model_mode_for_export(model, False):
  # trace, _ = torch.jit._get_trace_graph(model, args=(inputs,))
#     trace = torch.jit.trace(model, (inputs, ))
mo=model.eval()

In [ ]:
import pdir

In [ ]:
pdir(mo)

In [ ]:
mo

In [ ]:
a=['aten::Int', 'aten::ScalarImplicit', 'aten::abs', 'aten::add', 'aten::add_', 'aten::addmm', 'aten::arange', 'aten::bmm', 'aten::cat', 'aten::clone', 'aten::constant_pad_nd', 'aten::contiguous', 'aten::copy_', 'aten::cumsum', 'aten::div', 'aten::dropout', 'aten::einsum', 'aten::embedding', 'aten::eq', 'aten::expand_as', 'aten::fill_', 'aten::floor_divide', 'aten::full_like', 'aten::gather', 'aten::gelu', 'aten::index', 'aten::layer_norm', 'aten::le', 'aten::log', 'aten::lt', 'aten::masked_fill', 'aten::masked_fill_', 'aten::matmul', 'aten::max', 'aten::mean', 'aten::min', 'aten::mul', 'aten::mul_', 'aten::ne', 'aten::neg', 'aten::ones', 'aten::permute', 'aten::pow', 'aten::relu', 'aten::repeat', 'aten::reshape', 'aten::rsub', 'aten::select', 'aten::size', 'aten::slice', 'aten::softmax', 'aten::split', 'aten::sqrt', 'aten::squeeze', 'aten::stack', 'aten::sub', 'aten::sum', 'aten::t', 'aten::tanh', 'aten::to', 'aten::transpose', 'aten::triu', 'aten::type_as', 'aten::unsqueeze', 'aten::view', 'aten::where', 'aten::zeros', 'aten::zeros_like', 'prim::Constant', 'prim::GetAttr', 'prim::ListConstruct', 'prim::ListUnpack', 'prim::NumToTensor', 'prim::TupleConstruct', 'prim::TupleUnpack']

In [ ]:
for ai in a:
    print(f"'{ai}': '{ai.replace('::', '_')}',")

In [ ]:
a = torch.zeros(100, dtype=torch.long)

In [ ]:
import sys
sys.getsizeof(a)

In [ ]:
import time

start_times = dict()
end_times = dict()

def log_start_builder(name):
    def log_start(module, m_in):
        start_times[f'{name}:{module.__class__.__name__}'] = time.perf_counter()
    return log_start
def log_end_builder(name):
    def log_end(module, m_in, m_out):
        end_times[f'{name}:{module.__class__.__name__}'] = time.perf_counter()
#         print(name, module.__class__.__name__, 'end', time.perf_counter())
    return log_end

In [ ]:
for name, module in mo.named_modules():
    print(name, module.__class__.__name__)
    module.register_forward_pre_hook(log_start_builder(name))
    module.register_forward_hook(log_end_builder(name))

In [ ]:
trace = torch.jit.trace(model, inputs) # 1605233162.4006221

In [ ]:
for k, start in start_times.items():
    print(f'{k}, {(end_times[k]-start)*1000:.3f} ms, {start}, {end_times[k]}')

In [ ]:
graph = trace.inlined_graph

In [ ]:
graph

In [ ]:
ni=list(graph.nodes())
ni

In [ ]:
ni[0].output().node()

In [ ]:
ni[0].output()

In [ ]:
ni[72].output() # build all input leaf data nodes, then construct the opnode with input and output nodes

In [ ]:
ni[0]

In [ ]:
list(ni[0].outputs())

In [ ]:
list(ni[0].inputs())

In [ ]:
pn=ni[0].input().node()
pn

In [ ]:
gi=list(graph.inputs())
gi

In [ ]:
list(gi[0].node().outputs())[0].type()

In [ ]:
list(gi[1].node().outputs())[1].type().sizes()

In [ ]:
'encoder/encoder.layer.0/encoder.layer.0.attention/encoder.layer.0.attention.output'.split('/')[-1]

In [ ]:
print(pn)
print(gi[1].node())
pn==gi[0].node()

In [ ]:
[i.user for i in gi[0].uses()]

In [ ]:
go=list(graph.outputs())
go

In [ ]:
# sub_scopes='encoder.layer.0.attention.output.LayerNorm'.split('.')

In [ ]:
'true' if '' and len(''.split('.')) else 'false'

In [ ]:
sub_scopes='encoder.layer.0.attention.output.LayerNorm'.split('.')
for si in range(len(sub_scopes), 0, -1):
    p_scope = '.'.join(sub_scopes[0:si-1])
    child_scope = '.'.join(sub_scopes[0:si])
    print(si, p_scope, child_scope)